## Constants in BN256 

repo: https://github.com/privacy-scaling-explorations/halo2curves/tree/main/src/bn256 \
reference: https://github.com/Consensys/gnark-crypto/tree/v0.12.0/ecc/bn254


In [72]:
# Helper function, transfer a given integer into its little endian with 4 limbs, each limb is 64 bits
def u_64_little_endian(n):
    str_hex = hex(n)
    str_hex_without_0x = str_hex[2:]
    full_width_str = '0' * (64 - len(str_hex_without_0x)) + str_hex_without_0x
    assert len(full_width_str) == 64

    res = []
    for i in range(4):
        temp = '0x' + full_width_str[64 - 16 * (i + 1) : 64 - 16 * i]
        res.append(temp)
    return res

In [73]:
# Helper function, transfer a given integer into its NAF form
# def of NAF form: https://en.wikipedia.org/wiki/Non-adjacent_form
def naf_representation(n):
    naf = []
    while n > 0:
        if n % 2 == 1:
            # If n is odd, set the current digit as 2 - (n mod 4).
            digit = 2 - (n % 4)
            n -= digit
        else:
            # If n is even, set the current digit as 0.
            digit = 0
        naf.insert(0, digit)
        n //= 2
    return naf

# Example usage:
# decimal_number = 7
# naf = naf_representation(decimal_number)
# print(naf): [1, 0, 0, -1]

### The Base Field $\mathbb{F}_q$
seed $ U = 4965661367192848881$ \
the base field size $q = 36 U^4 + 36 U^3 + 24 U^2 + 6 U + 1 $

In [74]:
U = 4965661367192848881
q_int = 36 * U**4 + 36 * U**3 + 24 * U**2 + 6 * U + 1
hex(q_int)

'0x30644e72e131a029b85045b68181585d97816a916871ca8d3c208c16d87cfd47'

In [75]:
# https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/bn256/fq.rs#L34-L40 

u_64_little_endian(q_int)  

['0x3c208c16d87cfd47',
 '0x97816a916871ca8d',
 '0xb85045b68181585d',
 '0x30644e72e131a029']

In [76]:
Fq = GF(q_int)
Fq

Finite Field of size 21888242871839275222246405745257275088696311157297823662689037894645226208583

### The Montgomery constant used in $\mathbb{F}_q: R = 2^{256} \text{ mod } q$

In [77]:
R = Fq(2**256)
R

6350874878119819312338956282401532409788428879151445726012394534686998597021

### the Scalar Field $\mathbb{F}_r$
the scalar field size $r = 36 U^4 + 36 U^3 + 18 U^2 + 6 U+1$

In [78]:
r_int = 36 * U**4 + 36 * U**3 + 18 * U**2 + 6 * U + 1
hex(r_int)

'0x30644e72e131a029b85045b68181585d2833e84879b9709143e1f593f0000001'

In [79]:
# https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/bn256/fr.rs#L49-L55

u_64_little_endian(r_int)

['0x43e1f593f0000001',
 '0x2833e84879b97091',
 '0xb85045b68181585d',
 '0x30644e72e131a029']

In [80]:
Fr = GF(r_int)
Fr

Finite Field of size 21888242871839275222246405745257275088548364400416034343698204186575808495617

### the field $\mathbb{F}_{q^2}$

$\mathbb{F}_{q^2} = \mathbb{F}_q[u] = \mathbb{F}_q[X] / (X^2 + 1)$ where $X^2 + 1$ is irreducible in $\mathbb{F}_q[X]$

In [81]:
Rq = PolynomialRing(Fq, "x")
x = Rq.gen()
(x^2 + 1).is_irreducible()

True

In [82]:
# Construct the finite field Fq2 in u
Fq2.<u> = Fq.extension(x^2 + 1 )
Fq2

Finite Field in u of size 21888242871839275222246405745257275088696311157297823662689037894645226208583^2

In [83]:
# Check the identity
u**2 + 1 == 0

True

### The twisted curve coefficient or G2_B
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/bn256/curve.rs#L69-L82

In [84]:
3 * (u + 9)**(-1)

266929791119991161246907387137283842545076965332900288569378510910307636690*u + 19485874751759354771024239261021720505790618469301721065564631296452457478373

In [85]:
# c0
u_64_little_endian(19485874751759354771024239261021720505790618469301721065564631296452457478373)

['0x3267e6dc24a138e5',
 '0xb5b4c5e559dbefa3',
 '0x81be18991be06ac3',
 '0x2b149d40ceb8aaae']

In [86]:
# c1
u_64_little_endian(266929791119991161246907387137283842545076965332900288569378510910307636690)

['0xe4a2bd0685c315d2',
 '0xa74fa084e52d1852',
 '0xcd2cafadeed8fdf4',
 '0x009713b03af0fed4']

### the field $\mathbb{F}_{q^6}$
$\mathbb{F}_{q^6} = \mathbb{F}_{q^2}[v] = \mathbb{F}_{q^2}[X] / (X^3 - (u + 9))$ where $X^3 - (u + 9)$ is irreducible in $\mathbb{F}_{q^2}[X]$

In [87]:
Rq2 = PolynomialRing(Fq2, "x")
x = Rq2.gen()
(x^3 - (u + 9)).is_irreducible()

True

Construct the field $\mathbb{F}_{q^6}$ in $v$ by the polynomial quotient ring $\mathbb{F}_{p^2}[X] / (X^3 - (u + 9))$. \
Sagemath doesn't provide a method of constructing extention fields over non-prime base field.


In [88]:
Fq6.<v> = Fq2.extension(x^3 - (u + 9))
Fq6

Univariate Quotient Polynomial Ring in v over Finite Field in u of size 21888242871839275222246405745257275088696311157297823662689037894645226208583^2 with modulus v^3 + 21888242871839275222246405745257275088696311157297823662689037894645226208582*u + 21888242871839275222246405745257275088696311157297823662689037894645226208574

In [89]:
# Check Fq6 is indeed field
Fq6.is_field()

True

In [90]:
# Check the identity
v^3 - (u + 9) == 0

True

Note that the non-cubic residue $u + 9$ is not arbitrary here. It is closely related to the equation of the given curve:
$$E / \mathbb{F}_p: Y^2 = X^3 + 3$$
and the twisted curve where $\mathbb{G}_2$ is defined :
$$E' / \mathbb{F}_{p^2}: Y^2 = X^3 + 3/(u^2 + 9)$$
See the reference: https://github.com/Consensys/gnark-crypto/blob/v0.12.0/ecc/bn254/bn254.go#L23-L24 \
In fact, the choices of the extension fields $\mathbb{F}_{q^2}, \mathbb{F}_{q^{6}}, \mathbb{F}_{q^{12}}$ are used for the computation of pairing, where the role of the twisted curve is crucial. More precisely, the isomorphism is defined by
$$E' \rightarrow E: \quad (x, y) \mapsto (x \cdot (u + 9)^{1/3}, y \cdot (u + 9)^{1/2})$$
See more details in Section 3 in paper: http://indigo.ie/~mscott/twists.pdf

### the field $\mathbb{F}_q^{12}$
$\mathbb{F}_{q^{12}} = \mathbb{F}_{q^6}[w] = \mathbb{F}_{q^6}[X] / (X^2 - v)$ where $X^2 - v$ is irreducible in $\mathbb{F}_{q^6}[X]$ \
We may use the following constructor to construct $\mathbb{F}_{q^{12}}$ as usual, but it turns out too slow:


In [91]:
# Rq6 = PolynomialRing(Fq6, "x")
# x = Rq6.gen()
# (x^2 - v).is_irreducible()
# Fq12 = Rq6.quotient(x^2 - v)
# Fq12

Instead, let's check it manually. It suffices to prove $X^2 - v$ is irreducible over $\mathbb{F}_{q^6}[X]$, in other words
$v$ is not a quadratic residue over $\mathbb{F}_{q^6}$. Otherwise, there is an element $p(v) = a_0 + a_1 \cdot v + a_2 \cdot v^2 \in \mathbb{F}_{q^6}$ such that
$$p(v)^2 = v \text{ over } \mathbb{F}_{q^6}$$
Equivalently, there exists $p(X) = a_0 + a_1 \cdot X + a_2 \cdot X^2 \in \mathbb{F}_{q^2}[X]$ such that
$$ X^3 - (u + 9) \;| \; p(X)^2 - X \text{ over } \mathbb{F}_{q^2}[X]$$
It is clear that there is no such a $p(X) \in \mathbb{F}_{q^2}[X]$.


### The const FROBENIUS_COEFF_FQ6_C1[1] 
used in the pairing: \
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/bn256/engine.rs#L432 \
is generated in the following way:

In [92]:

(u + 9)**((q_int-1)/3)

10307601595873709700152284273816112264069230130616436755625194854815875713954*u + 21575463638280843010398324269430826099269044274347216827212613867836435027261

so the second component c0 = 21575463638280843010398324269430826099269044274347216827212613867836435027261, \
multiplied by the Montgomery const $R$, we get its representation in $\mathbb{F}_q$ \
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/bn256/fq6.rs#L449-L454:

In [93]:
u_64_little_endian(Fq(21575463638280843010398324269430826099269044274347216827212613867836435027261 * R))

['0xb5773b104563ab30',
 '0x347f91c8a9aa6454',
 '0x7a007127242e0991',
 '0x1956bcd8118214ec']

and the first component c1 = 10307601595873709700152284273816112264069230130616436755625194854815875713954, \
multiplied by the the Montgomery const $R$, we get its representation in $\mathbb{F}_q$ \
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/bn256/fq6.rs#L455-L460

In [94]:
u_64_little_endian(10307601595873709700152284273816112264069230130616436755625194854815875713954 * R)

['0x6e849f1ea0aa4757',
 '0xaa1c7b6d89f89141',
 '0xb6e713cdfae0ca3a',
 '0x26694fbb4e82ebc3']

### The const FROBENIUS_COEFF_FQ6_C1[2] 
used in the pairing: \
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/bn256/engine.rs#L440 \
is generated in the following way:

In [95]:
(u + 9)**((q_int**2-1)/3)

21888242871839275220042445260109153167277707414472061641714758635765020556616

so the second component c0 = 21888242871839275220042445260109153167277707414472061641714758635765020556616, \
multiplied by the Montgomery const $R$, we get its representation in $\mathbb{F}_q$ \
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/bn256/fq6.rs#L464-L470

In [96]:
u_64_little_endian(21888242871839275220042445260109153167277707414472061641714758635765020556616 * R)

['0x3350c88e13e80b9c',
 '0x7dce557cdb5e56b9',
 '0x6001b4b8b615564a',
 '0x2682e617020217e0']

and the first component c1 = 0, \
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/bn256/fq6.rs#L471

### The const XI_TO_Q_MINUS_1_OVER_2
used in the pairing\
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/bn256/engine.rs#L435 \
is generated in the following way:

In [97]:
(u + 9)**((q_int-1)/2)

3505843767911556378687030309984248845540243509899259641013678093033130930403*u + 2821565182194536844548159561693502659359617185244120367078079554186484126554

so the second component c0 = 2821565182194536844548159561693502659359617185244120367078079554186484126554, \
multiplied by the Montgomery const $R$, we get its representation in $\mathbb{F}_q$ \
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/bn256/engine.rs#L28-L33

In [98]:
u_64_little_endian(2821565182194536844548159561693502659359617185244120367078079554186484126554 * R)


['0xe4bbdd0c2936b629',
 '0xbb30f162e133bacb',
 '0x31a9d1b6f9645366',
 '0x253570bea500f8dd']

and the first component c1 = 3505843767911556378687030309984248845540243509899259641013678093033130930403, \
multiplied by the the Montgomery const $R$, we get its representation in $\mathbb{F}_q$ \
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/bn256/engine.rs#L34-L39

In [99]:
u_64_little_endian(3505843767911556378687030309984248845540243509899259641013678093033130930403 * R)


['0xa1d77ce45ffe77c7',
 '0x07affd117826d1db',
 '0x6d16bd27bb7edc6b',
 '0x2c87200285defecc']

### The const SIX_U_PLUS_2_NAF
used in the pairing\
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/bn256/engine.rs#L21-L25 \
is generated in the following way:

In [100]:
L = naf_representation(6*U + 2)
L[::-1]

[0,
 0,
 0,
 1,
 0,
 1,
 0,
 -1,
 0,
 0,
 -1,
 0,
 0,
 0,
 1,
 0,
 0,
 -1,
 0,
 -1,
 0,
 0,
 0,
 1,
 0,
 -1,
 0,
 0,
 0,
 0,
 -1,
 0,
 0,
 1,
 0,
 -1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 0,
 -1,
 0,
 1,
 0,
 -1,
 0,
 0,
 0,
 -1,
 0,
 -1,
 0,
 0,
 0,
 1,
 0,
 -1,
 0,
 1]